In [1]:
import pandas as pd

In [35]:
df = pd.read_csv('/home/workspace/data/GLDv2/train/train_custom.csv')

In [5]:
df.head()

,id,landmark_id,supercategory,description,language
0,4e7c3c4e083ee269,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
1,0ddc92c6353c019a,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
2,d500d704532092dc,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
3,8a80dbb49ccb966d,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
4,4a454571caa80175,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en


In [10]:
cut_df = df.iloc[:90:30, :]

In [12]:
cut_df.to_csv('/home/workspace/data/GLDv2/train/train_custom_demo.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549598 entries, 0 to 2549597
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   id             object
 1   landmark_id    int64 
 2   supercategory  object
 3   description    object
 4   language       object
dtypes: int64(1), object(4)
memory usage: 97.3+ MB


In [33]:
train_df = pd.read_csv('/home/workspace/data/GLDv2/train/train.csv') 

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132914 entries, 0 to 4132913
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   id           object
 1   url          object
 2   landmark_id  int64 
dtypes: int64(1), object(2)
memory usage: 94.6+ MB


In [9]:
train_df.head()

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474


In [39]:
train_clean_df = pd.read_csv('/home/workspace/data/GLDv2/train/train_clean.csv') 

In [5]:
train_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81313 entries, 0 to 81312
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   landmark_id  81313 non-null  int64 
 1   images       81313 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [8]:
train_clean_df.head()

,landmark_id,images
0,1,17660ef415d37059 92b6290d571448f6 cd41bf948edc...
1,7,25c9dfc7ea69838d 28b13f94a6f1f3c1 307d6584f473...
2,9,0193b65bb58d2c77 1a30a51a287ecf69 1f4e8ab1f1b2...
3,11,1a6cb1deed46bb17 1cc2c8fbc83e1a0c 2361b8da868c...
4,12,0a199c97c382b1ff 1492a5d344495391 290097bd36a6...


### train.csv처럼 이미지 1개당 랜드마크 1개로 매칭되게 수정

In [40]:
train_clean_df = (
    train_clean_df.assign(images=train_clean_df["images"].str.split())  # ①
      .explode("images")                        # ②
      .reset_index(drop=True)                   # 선택: 인덱스 초기화
)

In [13]:
train_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580470 entries, 0 to 1580469
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   landmark_id  1580470 non-null  int64 
 1   images       1580470 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.1+ MB


In [14]:
train_clean_df.head()

,landmark_id,images
0,1,17660ef415d37059
1,1,92b6290d571448f6
2,1,cd41bf948edc0340
3,1,fb09f1e98c6d2f70
4,7,25c9dfc7ea69838d


### images -> id로 변경, landmark_id와 id 열 위치 교체

In [41]:
# 1) 'images' → 'id' 로 열 이름 변경
train_clean_df = train_clean_df.rename(columns={"images": "id"})

# 2) 열 순서 재배치: id 먼저, landmark_id 뒤
train_clean_df = train_clean_df[["id", "landmark_id"] + [c for c in train_clean_df.columns if c not in ["id", "landmark_id"]]]


In [42]:
train_clean_df.head()

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7


### train_custom.csv의 supercategory, description, language 열 붙이기

In [43]:
train_clean_custom_df = pd.merge(train_clean_df, df[["id", "supercategory", "description", "language"]], on="id", how="left")

In [44]:
train_clean_custom_df

,id,landmark_id,supercategory,description,language
0,17660ef415d37059,1,park,Luitpoldpark is a public park in the Schwabing...,en
1,92b6290d571448f6,1,park,Luitpoldpark is a public park in the Schwabing...,en
2,cd41bf948edc0340,1,park,Luitpoldpark is a public park in the Schwabing...,en
3,fb09f1e98c6d2f70,1,park,Luitpoldpark is a public park in the Schwabing...,en
4,25c9dfc7ea69838d,7,multi-purpose hall,Die Sparkassen-Arena ist eine Mehrzweckhalle i...,de
...,...,...,...,...,...
1580465,72c3b1c367e3d559,203092,city,Hoa Lư was the capital of Vietnam from 968 to ...,en
1580466,7a6a2d9ea92684a6,203092,city,Hoa Lư was the capital of Vietnam from 968 to ...,en
1580467,9401fad4c497e1f9,203092,city,Hoa Lư was the capital of Vietnam from 968 to ...,en
1580468,aacc960c9a228b5f,203092,city,Hoa Lư was the capital of Vietnam from 968 to ...,en


In [45]:
train_clean_custom_df.to_csv('/home/workspace/data/GLDv2/train/train_clean_custom.csv', index=False)

In [1]:
import pandas as pd
train_custom_df = pd.read_csv('/home/workspace/data/GLDv2/train/train_custom.csv')
train_clean_custom_df = pd.read_csv('/home/workspace/data/GLDv2/train/train_clean_custom.csv')

In [8]:
train_clean_custom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580470 entries, 0 to 1580469
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   id             1580470 non-null  object
 1   landmark_id    1580470 non-null  int64 
 2   supercategory  1041805 non-null  object
 3   description    1041805 non-null  object
 4   language       1041805 non-null  object
dtypes: int64(1), object(4)
memory usage: 60.3+ MB


In [21]:
landmark_id_list = train_clean_custom_df[train_clean_custom_df["description"].isnull()]['landmark_id'].unique()

In [16]:
train_custom_df[train_custom_df['landmark_id'] == 203085]

,id,landmark_id,supercategory,description,language


In [12]:
train_category_df = pd.read_csv("/home/workspace/data/GLDv2/train/train_label_to_category.csv")

In [17]:
train_category_df[train_category_df['landmark_id'] == 203085]

,landmark_id,category
203085,203085,http://commons.wikimedia.org/wiki/Category:Chu...


In [14]:
train_hierarchical_df = pd.read_csv("/home/workspace/data/GLDv2/train/train_label_to_hierarchical.csv")

In [ ]:
train_hierarchical_df[train_hierarchical_df['landmark_id'] == 9]

0

In [ ]:
count = 0
for landmark_id in landmark_id_list:
    if len(train_hierarchical_df[train_hierarchical_df['landmark_id'] == landmark_id]) == 0:
        count += 1

print(count)

24872


In [28]:
count = 0
for landmark_id in landmark_id_list:
    if len(train_category_df[train_category_df['landmark_id'] == landmark_id]) == 0:
        count += 1

print(count)

0


In [31]:
train_clean_custom_df[train_clean_custom_df["description"].isnull()][["id", "landmark_id"]].to_csv('/home/workspace/data/GLDv2/train/train_clean_missing_description.csv', index=False)

In [32]:
train_clean_missing_description_df = pd.merge(train_clean_custom_df[train_clean_custom_df["description"].isnull()][["id", "landmark_id"]], train_category_df, on="landmark_id", how="left")

In [35]:
train_clean_missing_description_df.to_csv('/home/workspace/data/GLDv2/train/train_clean_missing_description.csv', index=False)

In [1]:
import pandas as pd
train_clean_missing_description_df = pd.read_csv('/home/workspace/data/GLDv2/train/train_clean_missing_description.csv')

In [5]:
train_clean_missing_description_df['landmark_id'].nunique()

35328

In [3]:
train_clean_missing_description_unique_df = (
    train_clean_missing_description_df.drop_duplicates(subset='landmark_id',           # 중복 판정 기준
                       keep='first')                   # 첫 번째 행만 남김
      .reset_index(drop=True)                          # 인덱스 재정렬(선택)
)

In [6]:
train_clean_missing_description_unique_df.to_csv('/home/workspace/data/GLDv2/train/train_clean_missing_description_unique.csv', index=False)

In [14]:
train_clean_missing_description_unique_df.iloc[:30, :].to_csv('/home/workspace/data/GLDv2/train/train_clean_missing_description_unique_demo.csv', index=False)